In [11]:
import pandas as pd
import numpy as np

import rasterio
from skimage.transform import resize
from skimage.transform import rotate
import os

import torch
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

#### Import Yield Data

In [12]:
# Load yield data
yield_data_path = "./combined_yield_data.csv"
yield_data = pd.read_csv(yield_data_path, parse_dates=['Date'], index_col='Date')

print(yield_data.head())
print("\nNumber of Yield Data Points: ", len(yield_data))
print("\nColumn Names:", yield_data.columns)

            Volume (Pounds)  Cumulative Volumne (Pounds)  Pounds/Acre
Date                                                                 
2012-01-02          23400.0                      23400.0          2.0
2012-01-03          26064.0                      49464.0          3.0
2012-01-04          32382.0                      81846.0          3.0
2012-01-05          69804.0                     151650.0          7.0
2012-01-06          18000.0                     169650.0          2.0

Number of Yield Data Points:  3970

Column Names: Index(['Volume (Pounds)', 'Cumulative Volumne (Pounds)', 'Pounds/Acre'], dtype='object')


#### Process Yield Data

In [13]:
# Define the typical strawberry growing season as a temporal mask (so that we only consider when strawberries are being grown)
def is_strawberry_season(date):
    return date.month in [3, 4, 5, 6, 7, 8, 9, 10]

# Filter yield data to only include dates within the strawberry growing season
yield_data = yield_data[yield_data.index.map(is_strawberry_season)]

print("Number of Yield Data Points:", len(yield_data))

Number of Yield Data Points: 2879


#### Resample to be on a weekly basis

In [14]:
# Resample yield data to weekly frequency
yield_data_weekly = yield_data.resample('W').agg({
    'Volume (Pounds)': 'sum',
    'Cumulative Volumne (Pounds)': 'last',
    'Pounds/Acre': 'mean'
})
yield_data_weekly['Cumulative Volumne (Pounds)'] = yield_data_weekly['Cumulative Volumne (Pounds)'].ffill()
yield_data_weekly['Cumulative Volumne (Pounds)'] = yield_data_weekly['Cumulative Volumne (Pounds)'].cummax()
yield_data_weekly.fillna(0, inplace=True)


#### Add cyclical features for seasonality

In [15]:
# Add time features to yield data
yield_data_weekly['month'] = yield_data_weekly.index.month
yield_data_weekly['day_of_year'] = yield_data_weekly.index.dayofyear

# Cyclical encoding for month
yield_data_weekly['month_sin'] = np.sin(2 * np.pi * yield_data_weekly['month'] / 12)
yield_data_weekly['month_cos'] = np.cos(2 * np.pi * yield_data_weekly['month'] / 12)

# Cyclical encoding for day of year
yield_data_weekly['day_of_year_sin'] = np.sin(2 * np.pi * yield_data_weekly['day_of_year'] / 365)
yield_data_weekly['day_of_year_cos'] = np.cos(2 * np.pi * yield_data_weekly['day_of_year'] / 365)

# Drop original time features
yield_data_weekly.drop(['month', 'day_of_year'], axis=1, inplace=True)

print("Yield data with time features:")
print(yield_data_weekly.head())

Yield data with time features:
            Volume (Pounds)  Cumulative Volumne (Pounds)  Pounds/Acre  \
Date                                                                    
2012-03-04         525753.0                    1785843.0    18.333333   
2012-03-11        2949534.0                    4735377.0    51.666667   
2012-03-18        4772268.0                    9507645.0    83.500000   
2012-03-25        3142314.0                   12649959.0    55.000000   
2012-04-01        6271398.0                   18921357.0    93.857143   

            month_sin     month_cos  day_of_year_sin  day_of_year_cos  
Date                                                                   
2012-03-04   1.000000  6.123234e-17         0.891981         0.452072  
2012-03-11   1.000000  6.123234e-17         0.939856         0.341571  
2012-03-18   1.000000  6.123234e-17         0.974100         0.226116  
2012-03-25   1.000000  6.123234e-17         0.994218         0.107381  
2012-04-01   0.866025 -5.

#### Yield Data Normalization

In [16]:
scaler = MinMaxScaler()
yield_data_weekly[['Volume (Pounds)', 'Cumulative Volumne (Pounds)']] = scaler.fit_transform(yield_data_weekly[['Volume (Pounds)', 'Cumulative Volumne (Pounds)']])

#### Load and Preprocess EVI Data

In [17]:
# Directory containing the pre-filtered EVI data
evi_data_dir = './landsat_evi_monterey_masked'

# List all files
evi_files = [os.path.join(evi_data_dir, f) for f in os.listdir(evi_data_dir) if f.endswith('.tiff')]

# Function to load EVI data
def load_evi_data(file_path):
    with rasterio.open(file_path) as src:
        data = src.read(1)
        return data

# Load EVI data
evi_data_dict = {}
for file in evi_files:
    try:
        date_str = os.path.basename(file).split('_')[3]
        date = pd.to_datetime(date_str, format='%Y%m%d') # Extract date from the file name
        evi_data = load_evi_data(file)
        evi_data_dict[date] = evi_data
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Define target shape (experimented with smaller; larger might perform better but might take longer to train)
target_shape = (512, 512)

# Function to resize and normalize image
def preprocess_image(image, target_shape):
    image_resized = resize(image, target_shape, anti_aliasing=True)
    return (image_resized - np.min(image_resized)) / (np.max(image_resized) - np.min(image_resized) + 1e-8)  # Adding a small value to prevent division by zero

# Preprocess all EVI images
evi_data_dict = {date: preprocess_image(data, target_shape) for date, data in evi_data_dict.items()}

print(f"Preprocessed EVI data for {len(evi_data_dict)} dates.")

Preprocessed EVI data for 83 dates.


#### Data Augmentation

In [18]:
# Function to augment image
def augment_image(image):
    # Apply random horizontal and vertical flips
    if np.random.rand() > 0.5:
        image = np.flipud(image)
    if np.random.rand() > 0.5:
        image = np.fliplr(image)
    # Apply random rotation
    angle = np.random.uniform(-30, 30)  # Rotate between -30 to 30 degrees
    image = rotate(image, angle, mode='reflect')
    return image

# Apply augmentation to EVI data
evi_data_dict_aug = {date: augment_image(data) for date, data in evi_data_dict.items()}

# Combine original and augmented EVI data
evi_data_dict_combined = {f"{date}_aug": data for date, data in evi_data_dict_aug.items()}
evi_data_dict_combined.update(evi_data_dict)

# Update evi_reference to include augmented data
time_index = pd.date_range(start=yield_data_weekly.index[0], end=yield_data_weekly.index[-1], freq='W')
evi_reference = [min(evi_data_dict.keys(), key=lambda d: abs(d - week_start)) for week_start in time_index]
evi_reference_aug = [f"{date}_aug" for date in evi_reference]

# Combine original and augmented references
evi_reference_combined = evi_reference + evi_reference_aug

print("Number of samples in augmented dataset:", len(evi_reference_combined))

Number of samples in augmented dataset: 1282


In [19]:
class CustomDataset(Dataset):
    def __init__(self, evi_data_dict, evi_reference, yield_data, sequence_length=4):
        self.evi_data_dict = evi_data_dict
        self.evi_reference = evi_reference
        self.yield_data = yield_data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.yield_data) - self.sequence_length + 1

    def __getitem__(self, idx):
        evi_sequence = []
        for I in range(self.sequence_length):
            evi_date = self.evi_reference[idx + I]
            evi_sequence.append(self.evi_data_dict[evi_date])
        evi_sequence = torch.tensor(evi_sequence, dtype=torch.float32).unsqueeze(1)
        yield_val = self.yield_data.iloc[idx + self.sequence_length - 1]['Volume (Pounds)']
        time_features = self.yield_data.iloc[idx + self.sequence_length - 1][['month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'Volume (Pounds)', 'Cumulative Volumne (Pounds)']].values
        return evi_sequence, torch.tensor(yield_val, dtype=torch.float32), torch.tensor(time_features, dtype=torch.float32)
    
# Create DataLoader
dataset = CustomDataset(evi_data_dict_combined, evi_reference_combined, yield_data_weekly)
train_indices, test_indices = train_test_split(np.arange(len(dataset)), test_size=0.2, random_state=42)

train_subset = torch.utils.data.Subset(dataset, train_indices)
test_subset = torch.utils.data.Subset(dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=4, shuffle=False)

#### Define the Model

In [20]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(CNNFeatureExtractor, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)  # Adjust dropout rate
        self.flattened_size = self._get_conv_output((1, *target_shape))
        self.fc1 = nn.Linear(self.flattened_size, 512)

    def _get_conv_output(self, shape):
        x = torch.rand(1, *shape)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        n_size = x.view(1, -1).size(1)
        return n_size

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout(x)
        x = x.view(-1, self.flattened_size)
        x = F.relu(self.fc1(x))
        return x

    
class HybridModel(nn.Module):
    def __init__(self, cnn_feature_extractor, lstm_hidden_size=64, lstm_layers=1):
        super(HybridModel, self).__init__()
        self.cnn = cnn_feature_extractor
        self.lstm = nn.LSTM(input_size=512, hidden_size=lstm_hidden_size, num_layers=lstm_layers, batch_first=True)
        self.fc1 = nn.Linear(lstm_hidden_size + 6, 64)  # Include space for time features and additional features
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x, time_features):
        batch_size, time_steps, C, H, W = x.size()
        c_in = x.view(batch_size * time_steps, C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, time_steps, -1)
        r_out, (h_n, c_n) = self.lstm(r_in)
        r_out = r_out[:, -1, :]
        x = torch.cat((r_out, time_features), dim=1) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### Initialize Function

In [21]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

# Instantiate model with weight decay regularization
cnn_feature_extractor = CNNFeatureExtractor()
model = HybridModel(cnn_feature_extractor)
model.apply(weights_init)  # Apply weight initialization

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

#### Training loop with early stopping

In [22]:
best_loss = float('inf')
patience = 3
trigger_times = 0
epochs = 50

for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    for i, (inputs, labels, time_features) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        outputs = model(inputs, time_features)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    scheduler.step()
    print(f'Epoch {epoch+1}, Loss: {epoch_loss}')

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'best_hybrid_model.pth')  # Save best model
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping!")
            break

100%|██████████| 128/128 [06:45<00:00,  3.17s/it]


Epoch 1, Loss: 0.0482490054948812


 82%|████████▏ | 105/128 [05:36<01:13,  3.19s/it]

In [ ]:
# Load the best model
model.load_state_dict(torch.load('best_hybrid_model.pth'))

# Extract features using the trained CNN and LSTM
features = []
labels = []
time_features_list = []
model.eval()  # Set model to evaluation mode

with torch.no_grad():
    for inputs, label, time_features in dataset:
        inputs = inputs.unsqueeze(0)  # Add batch dimension
        time_features_list.append(time_features.cpu().numpy())
        feature = model.cnn(inputs.view(-1, 1, target_shape[0], target_shape[1])).cpu().numpy().flatten()
        features.append(feature)
        labels.append(label.cpu().numpy())

features = np.array(features)
labels = np.array(labels)
time_features_list = np.array(time_features_list)
combined_features = np.concatenate((features, time_features_list), axis=1)

# Train a linear regression model
regressor = LinearRegression()
regressor.fit(combined_features, labels)

# Evaluate the linear regression model
predictions = regressor.predict(combined_features)
mse = mean_squared_error(labels, predictions)
print(f'Mean Squared Error: {mse}')

# Perform cross-validation
kf = KFold(n_splits=5)
mse_scores = []

for train_index, test_index in kf.split(combined_features):
    X_train, X_test = combined_features[train_index], combined_features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    regressor.fit(X_train, y_train)
    predictions = regressor.predict(X_test)
    mse_scores.append(mean_squared_error(y_test, predictions))

print(f'Cross-Validation MSE: {np.mean(mse_scores)}')

# Save model and data
torch.save(model.state_dict(), 'hybrid_model.pth')
np.save('features.npy', features)
np.save('labels.npy', labels)
np.save('time_features.npy', time_features_list)

Mean Squared Error: 0.013198168016970158
Cross-Validation MSE: 313.03131103515625
